Импорт библиотек

In [118]:
import requests as r
import pandas as pd
import datetime as dt
import numpy as np

Задание 3

In [138]:
#Подавление ошибки цепного назначения
pd.options.mode.chained_assignment = None

#Раскрываем все строки и столбцы
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Запрашиваем данные через GET запрос и преобразуем в json
visits_json = r.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01').json()
registrations_json = r.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01').json()

#Преобразовываем json в DataFrame
visits = pd.DataFrame(visits_json)
registrations = pd.DataFrame(registrations_json)

#Добавляем столбец с датой
visits['date'] = pd.to_datetime(visits['datetime']).dt.date
registrations['date'] = pd.to_datetime(registrations['datetime']).dt.date

#Выбираем дату последнего визита каждого пользователя
visits["datetime"] = pd.to_datetime(visits["datetime"], format="%a, %d %b %Y %H:%M:%S %Z")
visits["last_datetime"] = visits.groupby('visit_id')["datetime"].transform('max')
last_visits = visits[visits['datetime'] == visits['last_datetime']]

#Маркируем ботов
# def is_bot(row):
#     return 1 if row['user_agent'].str.contains('bot') else 0

# last_visits['is_bot'] = last_visits['user_agent'].apply(is_bot, axis = 1)

last_visits['is_bot'] = np.where(last_visits['user_agent'].str.contains('bot'), 'yes', 'no')

#Группируем и объединяем данные
visits_and_registrations = pd.concat([
    last_visits[last_visits['is_bot'] == 'no'].groupby(['date', 'platform']).agg({'visit_id': 'count'}),
    registrations.groupby(['date', 'platform']).agg({'user_id': 'count'})
], axis=1)

#Переименовываем столбцы и заполняем нули
visits_and_registrations.rename(columns = {'visit_id':'visits', 'user_id':'registrations'}, inplace = True )
visits_and_registrations['registrations'] = visits_and_registrations['registrations'].fillna(0)

#Считаем конверсию
visits_and_registrations['conversion'] = round(visits_and_registrations['registrations']/visits_and_registrations['visits']*100, 0)

#Сохраняем в формате json
visits_and_registrations.to_json('conversion.json', orient='columns')

print(visits_and_registrations)


                     visits  registrations  conversion
date       platform                                   
2023-03-01 android      264            218        83.0
           ios           78             67        86.0
           web          333             78        23.0
2023-03-02 android      106             91        86.0
           ios          103             76        74.0
           web          462             52        11.0
2023-03-03 android      251            218        87.0
           ios           57             54        95.0
           web         1221             66         5.0
2023-03-04 android      193            119        62.0
           ios          117             86        74.0
           web         1773             73         4.0
2023-03-05 android      287            209        73.0
           ios          110            101        92.0
           web         1924             52         3.0
2023-03-06 android      322            282        88.0
          